In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving CONCCOTP.DBF to CONCCOTP.DBF


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving DEVENGAD.DBF to DEVENGAD.DBF


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving SICOP.csv to SICOP.csv


In [ ]:
sicop = pd.read_csv('SICOP.csv', encoding='latin-1')



In [ ]:
mes=int(input("Ingrese el numero de mes: "))

Ingrese el numero de mes: 2


In [ ]:
!pip install dbfread


In [ ]:
from dbfread import DBF


# RECURSOS PROPIOS

In [ ]:
archivo_dbf = 'DEVENGAD.DBF'

# Specify the encoding when opening the DBF file
tabla_dbf = DBF(archivo_dbf, encoding='latin-1')

# Convertirlo a un DataFrame de pandas para facilitar la manipulación
import pandas as pd
devengad = pd.DataFrame(iter(tabla_dbf))



In [ ]:
#Filtrar recursos propios
devengad = devengad[devengad['TR'] == "2"]




In [ ]:
# Convertir la columna 'edad' a tipo int
devengad['PTDA'] = devengad['PTDA'].astype(int)

# Verificar el tipo de datos después de la conversión
print(devengad.dtypes)


TR       object
ACTI     object
PTDA      int64
NDOC     object
MCO     float64
MDE     float64
MPA     float64
MTO     float64
RFC      object
DESC     object
dtype: object


In [ ]:
#Ordenar
devengad.sort_values(by=['NDOC', 'ACTI','PTDA'], ascending=[True, True, True], inplace=True)

# Eliminar filas donde la columna empieza con "SUF-"
devengad = devengad[~devengad['NDOC'].str.startswith('SUF', na=False)]

devengad = devengad.reset_index(drop=True)

In [ ]:
archivo_dbf_2 = 'CONCCOTP.DBF'

# Specify the encoding when opening the DBF file
tabla_dbf_2 = DBF(archivo_dbf_2, encoding='latin-1')

# Convertirlo a un DataFrame de pandas para facilitar la manipulación
import pandas as pd
conccotp = pd.DataFrame(iter(tabla_dbf_2))



In [ ]:
# Reemplazar NaN por 0 solo en las columnas 'edad' y 'salario'
conccotp[['M01', 'M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']] = conccotp[['M01', 'M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']].fillna(0)



In [ ]:
#Quitar signos negativos
conccotp[['M01', 'M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']] = conccotp[['M01', 'M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12']].abs()


In [ ]:
#Ordenar
conccotp.sort_values(by=['NDOC', 'ACTI','PTDA'], ascending=[True, True, True], inplace=True)

# Eliminar filas donde la columna empieza con "SUF-"
conccotp = conccotp[~conccotp['NDOC'].str.startswith('SUF', na=False)]

conccotp = conccotp.reset_index(drop=True)

In [ ]:


# Generar la lista de columnas que se deben sumar según el mes
columnas_a_sumar = [f'M{str(i).zfill(2)}' for i in range(1, mes+1)]

# Sumar las columnas seleccionadas
conccotp['total_calendario'] = conccotp[columnas_a_sumar].sum(axis=1)



In [ ]:
#Hacer columna total pagado
devengad['total_pagado']=devengad['MDE']+devengad['MPA']

In [ ]:
#Unir el df devengad y conccotp
sifin_318=pd.merge(devengad, conccotp, left_index=True, right_index=True, how='inner')


In [ ]:
#Quedarme con las columnas que me interesan
sifin_318 = sifin_318[['TR_x', 'ACTI_x', 'PTDA_x','M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','total_pagado','total_calendario']]



In [ ]:
#Hacer la columna del totalote
sifin_318['totalote']=sifin_318['total_calendario']-sifin_318['total_pagado']

In [ ]:
#Hacer una pivot table

pivot_318=pd.pivot_table(sifin_318,
               index=['ACTI_x','PTDA_x'],
               values=['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','totalote'],
               aggfunc='sum')


In [ ]:
#Sustituye la columna del mes actual por la columna del totalote, además pone las columnas de meses anteriores llenando de ceros

columnas_anteriores = [f'M{str(m).zfill(2)}' for m in range(1, mes)]

columna_actual = ['totalote']

columnas_posteriores = [f'M{str(m).zfill(2)}' for m in range(mes+1, 13)]

columnas = columnas_anteriores + columna_actual + columnas_posteriores

final_318 = pivot_318[columnas]

for col in columnas_anteriores:
    if col in final_318.columns:
        final_318[col] = 0







/tmp/ipython-input-1750421932.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_318[col] = 0


In [ ]:
#Resetea indices de la pivot table para que se repitan etiquetas de la actividad
pivot_318_1 = final_318.reset_index()

In [ ]:
#Hace columnas para poder traer lo mismo que en SICOP
pivot_318_1['Tipo de Gasto'] = 1
pivot_318_1['Fuente de Financiamiento'] = 4
pivot_318_1['Entidad Federativa'] = 9
pivot_318_1['Clave de Cartera'] = 0
pivot_318_1['Periodicidad'] = 'M'
pivot_318_1['CVE_PERIODO'] = np.nan


In [ ]:
#Pone la estructura programatica

pivot_318_1['FINALIDAD'] = pd.Series(dtype='object')
pivot_318_1['FUNCION'] = pd.Series(dtype='object')
pivot_318_1['SUBFUNCION'] = pd.Series(dtype='object')
pivot_318_1['REASIGNACION'] = pd.Series(dtype='object')
pivot_318_1['ACTIV_INSTIT'] = pd.Series(dtype='object')
pivot_318_1['Modalidad'] = pd.Series(dtype='object')
pivot_318_1['Programa Presupuestario'] = pd.Series(dtype='object')

#E10
valores_e10 = ['E10C1', 'E10C2', 'E10F1','E10F2','E10F3','E10F4','E10F5','E10SP']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_e10),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['2', '3', '3', '0', '19', 'E', '10']

#E22
valores_e22 = ['E2203', 'E2204', 'E2205','E2206','E2208','E22SP']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_e22),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['3', '8', '1', '0', '24', 'E', '22']

#E23
valores_e23 = ['E2301', 'E2303', 'E2304','E2305','E23B2','E23GS','E23M1','E23MM','E23SP']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_e23),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['2', '3', '2', '0', '18', 'E', '23']

#obra
valores_k = ['K0811', 'K1164']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_k),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['2', '3', '3', '0', '20', 'K', '11']

#MIA2
valores_m2 = ['M02SP', 'M1AD2']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_m2),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['2', '3', '2', '0', '2', 'M', '1']

#MIA4
valores_m4 = ['M1AD4']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_m4),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['2', '3', '4', '0', '2', 'M', '1']



#:O01
valores_o01 = ['O01AU', 'O01SP']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_o01),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['1', '3', '4', '0', '1', 'O', '1']

#P16
valores_p16 = ['P16MJ']

pivot_318_1.loc[
    pivot_318_1['ACTI_x'].isin(valores_p16),
    ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario']
] = ['2', '3', '4', '0', '14', 'P', '16']


In [ ]:
pivot_318_1

# RECURSOS FISCALES

In [ ]:
#Filtra el df segun las columnas que necesito
sicop=sicop[['MES_RESPALDO','AÑO','ID_RAMO', 'ID_UNIDAD', 'FINALIDAD','FUNCION',
'SUBFUNCION',
'REASIGNACION',
'ACTIV_INSTIT',
'PROGRAMA_PRESUPUESTARIO',
'CAPITULO',
'CONCEPTO',
'PARTIDA_GENERICA',
'PARTIDA_ESPECIFICA',
'TIPO_GASTO',
'FUENTE_FINANCIAMIENTO',
'ENTIDAD_FEDERATIVA',
'CLAVE_CARTERA',
'CC',
'CONTROL_OPERATIVO',
'PLURIANUAL',
'ORGANISMO_FINANCIADOR',
'AUX1',
'AUX2',
'AUX3','COMPROMETIDO',
'COEN',
'COFE',
'COMR',
'COAB',
'COMY',
'COJN',
'COJL',
'COAG',
'COSE',
'COOC',
'CONO',
'CODI']]

In [ ]:
#Extraemos datos de la columna programa presupuestario para sacar la modalidad
sicop['Modalidad'] = sicop['PROGRAMA_PRESUPUESTARIO'].str[:1]


In [ ]:
#Extraemos datos de la columna programa presupuestario para sacar el programa presupuestario especifico
sicop['Programa Presupuestario'] = sicop['PROGRAMA_PRESUPUESTARIO'].str[2:]

In [ ]:
#Cambiamos el tipo de dato de las columnas
columnas = ['CAPITULO', 'CONCEPTO', 'PARTIDA_GENERICA','PARTIDA_ESPECIFICA','CC']  # columnas que quieres convertir

sicop[columnas] = sicop[columnas].astype(str)


In [ ]:
#Concatenamos para sacar la partida presupuestal
sicop['Objeto del Gasto']=sicop['CAPITULO']+sicop['CONCEPTO']+sicop['PARTIDA_GENERICA']+sicop['CC']+sicop['PARTIDA_ESPECIFICA']

In [ ]:
#Cambiamos el tipo de dato de la partida presupuestal
sicop['Objeto del Gasto'] = sicop['Objeto del Gasto'].astype(int)

In [ ]:
#Cambiamos las aprtidas que siempre salen mal
sicop.loc[sicop['Objeto del Gasto'] == 134011, 'Objeto del Gasto'] = 13411
sicop.loc[sicop['Objeto del Gasto'] == 134010, 'Objeto del Gasto'] = 13410


In [ ]:
#Hacemos nuestra pivot table
pivot_sicop=pd.pivot_table(sicop,
               index=['FINALIDAD','FUNCION','SUBFUNCION','REASIGNACION','ACTIV_INSTIT','Modalidad','Programa Presupuestario','Objeto del Gasto','TIPO_GASTO','FUENTE_FINANCIAMIENTO','ENTIDAD_FEDERATIVA','CLAVE_CARTERA'],
               values=['COEN','COFE','COMR','COAB','COMY','COJN','COJL','COAG','COSE','COOC','CONO','CODI','COMPROMETIDO'],
               aggfunc='sum')

In [ ]:
#Reseteamos los indices y cambiamos el orden de las columnas
pivot_sicop_1 = pivot_sicop.reset_index()[['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario', 'Objeto del Gasto', 'TIPO_GASTO', 'FUENTE_FINANCIAMIENTO', 'ENTIDAD_FEDERATIVA', 'CLAVE_CARTERA', 'COEN', 'COFE', 'COMR', 'COAB', 'COMY', 'COJN', 'COJL', 'COAG', 'COSE', 'COOC', 'CONO', 'CODI', 'COMPROMETIDO']]


In [ ]:
pivot_sicop_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   FINALIDAD                144 non-null    int64  
 1   FUNCION                  144 non-null    int64  
 2   SUBFUNCION               144 non-null    int64  
 3   REASIGNACION             144 non-null    int64  
 4   ACTIV_INSTIT             144 non-null    int64  
 5   Modalidad                144 non-null    object 
 6   Programa Presupuestario  144 non-null    object 
 7   Objeto del Gasto         144 non-null    int64  
 8   TIPO_GASTO               144 non-null    int64  
 9   FUENTE_FINANCIAMIENTO    144 non-null    int64  
 10  ENTIDAD_FEDERATIVA       144 non-null    int64  
 11  CLAVE_CARTERA            144 non-null    object 
 12  Periodicidad             144 non-null    object 
 13  CVE_PERIODO              0 non-null      float64
 14  COEN                     1

In [ ]:
columna_antes = 'COEN'
pos = pivot_sicop_1.columns.get_loc(columna_antes)

pivot_sicop_1.insert(pos, 'Periodicidad', 'M')


In [ ]:
columna_antes = 'COEN'
pos = pivot_sicop_1.columns.get_loc(columna_antes)

pivot_sicop_1.insert(pos, 'CVE_PERIODO', np.nan)


In [ ]:
pivot_sicop_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   FINALIDAD                144 non-null    int64  
 1   FUNCION                  144 non-null    int64  
 2   SUBFUNCION               144 non-null    int64  
 3   REASIGNACION             144 non-null    int64  
 4   ACTIV_INSTIT             144 non-null    int64  
 5   Modalidad                144 non-null    object 
 6   Programa Presupuestario  144 non-null    object 
 7   Objeto del Gasto         144 non-null    int64  
 8   TIPO_GASTO               144 non-null    int64  
 9   FUENTE_FINANCIAMIENTO    144 non-null    int64  
 10  ENTIDAD_FEDERATIVA       144 non-null    int64  
 11  CLAVE_CARTERA            144 non-null    object 
 12  Periodicidad             144 non-null    object 
 13  CVE_PERIODO              0 non-null      float64
 14  COEN                     1

#UNION

In [ ]:
columnas_sifin_2 = ['FINALIDAD', 'FUNCION', 'SUBFUNCION', 'REASIGNACION', 'ACTIV_INSTIT', 'Modalidad', 'Programa Presupuestario',
            'PTDA_x', 'Tipo de Gasto','Fuente de Financiamiento','Entidad Federativa','Clave de Cartera','Periodicidad',
            'CVE_PERIODO']

columnas_anteriores = [f'M{str(m).zfill(2)}' for m in range(1, mes)]

columna_actual = ['totalote']

columnas_posteriores = [f'M{str(m).zfill(2)}' for m in range(mes+1, 13)]

columnas = columnas_sifin_2 + columnas_anteriores + columna_actual + columnas_posteriores

sifin = pivot_318_1[columnas]




In [ ]:
sifin['COMPROMETIDO'] = sifin.iloc[:, 14:26].sum(axis=1)


In [ ]:
sifin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   FINALIDAD                 55 non-null     object 
 1   FUNCION                   55 non-null     object 
 2   SUBFUNCION                55 non-null     object 
 3   REASIGNACION              55 non-null     object 
 4   ACTIV_INSTIT              55 non-null     object 
 5   Modalidad                 55 non-null     object 
 6   Programa Presupuestario   55 non-null     object 
 7   PTDA_x                    55 non-null     int64  
 8   Tipo de Gasto             55 non-null     int64  
 9   Fuente de Financiamiento  55 non-null     int64  
 10  Entidad Federativa        55 non-null     int64  
 11  Clave de Cartera          55 non-null     int64  
 12  Periodicidad              55 non-null     object 
 13  CVE_PERIODO               0 non-null      float64
 14  M01         

In [ ]:

# Fuerza a df2 a usar el esquema (nombres y orden de columnas) de df1
df2_reformado = sifin.copy()
df2_reformado.columns = pivot_sicop_1.columns  # Reemplaza los nombres de columnas de df2 con los de pivot_sicop_1

# Ahora puedes concatenarlos
df_final = pd.concat([pivot_sicop_1, df2_reformado], ignore_index=True)


In [ ]:
df_final

,FINALIDAD,FUNCION,SUBFUNCION,REASIGNACION,ACTIV_INSTIT,Modalidad,Programa Presupuestario,Objeto del Gasto,TIPO_GASTO,FUENTE_FINANCIAMIENTO,...,COAB,COMY,COJN,COJL,COAG,COSE,COOC,CONO,CODI,COMPROMETIDO
0,1,3,4,0,1,O,01,11301,1,1,...,380570.0,357387.0,312138.0,390290.0,401637.0,527670.0,697584.0,755088.19,0.0,4175550.19
1,1,3,4,0,1,O,01,13201,1,1,...,0.0,0.0,37567.0,0.0,0.0,1323.0,0.0,165590.00,0.0,204480.00
2,1,3,4,0,1,O,01,13202,1,1,...,0.0,0.0,0.0,0.0,0.0,327817.0,0.0,325566.00,0.0,653383.00
3,1,3,4,0,1,O,01,13301,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3269.34
4,1,3,4,0,1,O,01,13404,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63777.48,0.0,63777.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1,3,4,0,1,O,1,32301,1,4,...,283.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1132.00
195,1,3,4,0,1,O,1,33801,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
196,1,3,4,0,1,O,1,35801,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,905.08
197,2,3,4,0,14,P,16,25301,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   FINALIDAD                199 non-null    object 
 1   FUNCION                  199 non-null    object 
 2   SUBFUNCION               199 non-null    object 
 3   REASIGNACION             199 non-null    object 
 4   ACTIV_INSTIT             199 non-null    object 
 5   Modalidad                199 non-null    object 
 6   Programa Presupuestario  199 non-null    object 
 7   Objeto del Gasto         199 non-null    int64  
 8   TIPO_GASTO               199 non-null    int64  
 9   FUENTE_FINANCIAMIENTO    199 non-null    int64  
 10  ENTIDAD_FEDERATIVA       199 non-null    int64  
 11  CLAVE_CARTERA            199 non-null    object 
 12  Periodicidad             199 non-null    object 
 13  CVE_PERIODO              0 non-null      float64
 14  COEN                     1

In [ ]:
#Hacemos nuestra pivot table
pivot_final=pd.pivot_table(df_final,
               index=['FINALIDAD','FUNCION','SUBFUNCION','REASIGNACION','ACTIV_INSTIT','Modalidad','Programa Presupuestario','Objeto del Gasto','TIPO_GASTO','FUENTE_FINANCIAMIENTO','ENTIDAD_FEDERATIVA','CLAVE_CARTERA'],
               values=['COEN','COFE','COMR','COAB','COMY','COJN','COJL','COAG','COSE','COOC','CONO','CODI','COMPROMETIDO'],
               aggfunc='sum')

In [ ]:
pivot_final.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 179 entries, (1, 3, 4, 0, 1, 'O', '01', np.int64(11301), np.int64(1), np.int64(1), np.int64(9), '00000000000') to ('3', '8', '1', '0', '24', 'E', '22', np.int64(35801), np.int64(1), np.int64(4), np.int64(9), 0)
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   COAB          179 non-null    float64
 1   COAG          179 non-null    float64
 2   CODI          179 non-null    float64
 3   COEN          179 non-null    int64  
 4   COFE          179 non-null    float64
 5   COJL          179 non-null    float64
 6   COJN          179 non-null    float64
 7   COMPROMETIDO  179 non-null    float64
 8   COMR          179 non-null    float64
 9   COMY          179 non-null    float64
 10  CONO          179 non-null    float64
 11  COOC          179 non-null    float64
 12  COSE          179 non-null    float64
dtypes: float64(12), int64(1)
memory usage: 21.8+ KB


In [ ]:
pivot_final.reset_index(inplace=True)

In [ ]:
pivot_final

,FINALIDAD,FUNCION,SUBFUNCION,REASIGNACION,ACTIV_INSTIT,Modalidad,Programa Presupuestario,Objeto del Gasto,TIPO_GASTO,FUENTE_FINANCIAMIENTO,...,COEN,COFE,COJL,COJN,COMPROMETIDO,COMR,COMY,CONO,COOC,COSE
0,1,3,4,0,1,O,01,11301,1,1,...,0,0.0,390290.0,312138.0,4175550.19,353186.00,357387.0,755088.19,697584.0,527670.0
1,1,3,4,0,1,O,01,13201,1,1,...,0,0.0,0.0,37567.0,204480.00,0.00,0.0,165590.00,0.0,1323.0
2,1,3,4,0,1,O,01,13202,1,1,...,0,0.0,0.0,0.0,653383.00,0.00,0.0,325566.00,0.0,327817.0
3,1,3,4,0,1,O,01,13301,1,1,...,0,0.0,0.0,0.0,3269.34,3269.34,0.0,0.00,0.0,0.0
4,1,3,4,0,1,O,01,13404,1,1,...,0,0.0,0.0,0.0,63777.48,0.00,0.0,63777.48,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,2,3,4,0,2,M,1,35801,1,4,...,0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0
175,3,8,1,0,24,E,22,27101,1,4,...,0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0
176,3,8,1,0,24,E,22,32301,1,4,...,0,1636.0,0.0,0.0,3272.00,818.00,0.0,0.00,0.0,0.0
177,3,8,1,0,24,E,22,33801,1,4,...,0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0


In [ ]:
final = pivot_final[['FINALIDAD','FUNCION','SUBFUNCION','REASIGNACION','ACTIV_INSTIT','Modalidad','Programa Presupuestario','Objeto del Gasto','TIPO_GASTO','FUENTE_FINANCIAMIENTO','ENTIDAD_FEDERATIVA','CLAVE_CARTERA','COEN','COFE','COMR','COAB','COMY','COJN','COJL','COAG','COSE','COOC','CONO','CODI','COMPROMETIDO']]

In [ ]:
final

,FINALIDAD,FUNCION,SUBFUNCION,REASIGNACION,ACTIV_INSTIT,Modalidad,Programa Presupuestario,Objeto del Gasto,TIPO_GASTO,FUENTE_FINANCIAMIENTO,...,COAB,COMY,COJN,COJL,COAG,COSE,COOC,CONO,CODI,COMPROMETIDO
0,1,3,4,0,1,O,01,11301,1,1,...,380570.0,357387.0,312138.0,390290.0,401637.0,527670.0,697584.0,755088.19,0.0,4175550.19
1,1,3,4,0,1,O,01,13201,1,1,...,0.0,0.0,37567.0,0.0,0.0,1323.0,0.0,165590.00,0.0,204480.00
2,1,3,4,0,1,O,01,13202,1,1,...,0.0,0.0,0.0,0.0,0.0,327817.0,0.0,325566.00,0.0,653383.00
3,1,3,4,0,1,O,01,13301,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3269.34
4,1,3,4,0,1,O,01,13404,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63777.48,0.0,63777.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,2,3,4,0,2,M,1,35801,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
175,3,8,1,0,24,E,22,27101,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
176,3,8,1,0,24,E,22,32301,1,4,...,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3272.00
177,3,8,1,0,24,E,22,33801,1,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
